In [18]:
import requests
import random
from tkinter import *
import json

In [20]:
url = "https://the-trivia-api.com/api/questions?categories=film_and_tv&limit=10&region=US&difficulty=easy"
get_request = requests.get(url).json()

root = Tk()
root.title("Trivia")
root.geometry("1000x800")

#canvas
canvas = Canvas(root, width=800, height=250, bg="khaki1")
canvas.place(relx=.5, rely=.5,anchor= CENTER)

game_display = canvas.create_text(200, 75, text="", fill="black", font=('Helvetica 15 bold'))

feedback = Label(root, text="", pady=10, font=("arial", 15, "bold"))

user_answer = StringVar()
user_answer.set("")

score = 0
num_questions = len(get_request)

def generate_qlist(json):
    question_list = []
    for i in json:
        i = i["question"]
        question_list.append(i)
    return question_list

def gen_question(list_):
    choice = random.choice(list_)
    return choice

def generate_answers(json, question): 
    option_list = []
    for i in get_request:
        if i["question"] == question:
            cor = i["correctAnswer"]
            option_list.append(cor)
            for incor in i["incorrectAnswers"]:
                option_list.append(incor)
    random.shuffle(option_list)
    return option_list

def ask_question(question, answers):
    canvas.itemconfig(game_display, text=question, width=300)

def gen_radiobuttons(answers):
    y_pos = 100
    for i in answers:
        rad_btn = Radiobutton(canvas, text=i, variable=user_answer, command = eval_score, value=i, font=('Helvetica 10 bold'))
        rad_btn.place(x=50, y=y_pos)
        y_pos+=20

def eval_score():
    answer = user_answer.get()
    for i in get_request:
        if i["question"] == current_question:
            if i["correctAnswer"] == answer:
                feedback["text"] = "Correct!"
                feedback.place(x=400, y=500)
                global score
                score += 10
                update_score()
                
            else:
                feedback["text"] = "Incorrect :("
                feedback.place(x=400, y=500)

def update_score():
    global score
    score_label = Label(root, text="Score: " + str(score), font=("arial", 15, "bold"))
    score_label["text"] = "Score: " + str(score)
    score_label.place(x=700, y=30)
    next_question()

q_list = generate_qlist(get_request)

def next_question():
    global current_question, num_questions
    global current_question
    current_question = gen_question(q_list)
    ans_list = generate_answers(get_request, current_question)
    ask_question(current_question, ans_list)
    gen_radiobuttons(ans_list)
    feedback["text"] = ""
    user_answer.set("")

    num_questions -= 1
    if num_questions == 0:
        end_game()

def end_game():
    canvas.delete(ALL)
    end_text = "You got " + str(score) + " out of " + str(len(get_request) * 10) + " correct!"
    end_label = Label(root, text=end_text, font=("arial", 20, "bold"))
    end_label.pack(pady=200)

def main():
    global current_question
    current_question = gen_question(q_list)
    ans_list = generate_answers(get_request, current_question)
    ask_question(current_question, ans_list)
    gen_radiobuttons(ans_list)
    
next_button = Button(root, text="Next Question", font=("times new roman", 12), command=next_question)
next_button.place(x=400, y=200)

main()
root.mainloop()

#https://www.freecodecamp.org/news/how-to-create-a-gui-quiz-application-using-tkinter-and-open-trivia-db